<a href="https://colab.research.google.com/github/Wasabi-Bobby/MachineLearningHomework/blob/master/KerasLogisticRegressionProblem5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Pretty much taken from the notes that were done in class and posted on github
from keras.datasets import mnist

(train_images_original, train_labels_original), (test_images_original, test_labels_original) = mnist.load_data()

train_images = train_images_original.reshape((60000, 28 * 28))
train_images = train_images.astype('float32') / 255

test_images = test_images_original.reshape((10000, 28 * 28))
test_images = test_images.astype('float32') / 255

Using TensorFlow backend.


11493376/11490434 [==============================] - 0s 0us/step


In [0]:
from keras.utils import to_categorical

train_labels = to_categorical(train_labels_original)
test_labels = to_categorical(test_labels_original)

In [0]:
# Flag is set to 0 if the current object is not continuous with white space, 1 if it is  
def DFS_whitespace(x, y, flag):
    if x < 0 or x > 27 or y < 0 or y > 27:
      return 0
    if visited_points[x][y] == 1:
      return 0
    if visited_points[x][y] == 0:
      visited_points[x][y] = 1
      DFS_whitespace(x+1, y, 1)
      DFS_whitespace(x-1, y, 1)
      DFS_whitespace(x, y+1, 1)
      DFS_whitespace(x, y-1, 1)
      if flag == 0:
        return 1
      return 0
    
def number_whitespace_count(number):
  if number == 0:
    return 2
  if number == 1:
    return 1
  if number == 2:
    return 1
  if number == 3:
    return 1
  if number == 4:
    return 2
  if number == 5:
    return 1
  if number == 6:
    return 2
  if number == 7:
    return 1
  if number == 8:
    return 3
  if number == 9:
    return 2

In [0]:
import numpy as np

train_images_rounded = np.copy(train_images)
visited_points = np.zeros((28, 28))
whitespace_train_matrix = np.zeros((60000, 10))

for i in range(len(train_images_rounded)):
  visited_points = np.zeros((28, 28))
  
  for j in range(len(train_images_rounded[i])):
    if train_images_rounded[i][j] > 0.5:
      train_images_rounded[i][j] = 1.0
      x = y = 0
      x = (int)(j/28)
      y = (int)(j%28)
      visited_points[x][y] = 1
    else:
      train_images_rounded[i][j] = 0.0
  
  white_space_current_count = 0
  
  for j in range(28):
    for k in range(28):
      white_space_current_count += DFS_whitespace(j, k, 0)
      
  for j in range(10):
    if white_space_current_count == number_whitespace_count(j):
      whitespace_train_matrix[i][j] = 1
    else:
      whitespace_train_matrix[i][j] = 0

In [0]:
import numpy as np

test_images_rounded = np.copy(test_images)
visited_points = np.zeros((28, 28))
whitespace_test_matrix = np.zeros((10000, 10))

for i in range(len(test_images_rounded)):
  visited_points = np.zeros((28, 28))
  
  for j in range(len(test_images_rounded[i])):
    if test_images_rounded[i][j] > 0.5:
      test_images_rounded[i][j] = 1.0
      x = y = 0
      x = (int)(j/28)
      y = (int)(j%28)
      visited_points[x][y] = 1
    else:
      test_images_rounded[i][j] = 0.0
  
  white_space_current_count = 0
  
  for j in range(28):
    for k in range(28):
      white_space_current_count += DFS_whitespace(j, k, 0)
  
  for j in range(10):
    if white_space_current_count == number_whitespace_count(j):
      whitespace_test_matrix[i][j] = 1
    else:
      whitespace_test_matrix[i][j] = 0

In [0]:
whitespace_test_matrix = whitespace_test_matrix.reshape((10000, 10))
whitespace_train_matrix = whitespace_train_matrix.reshape((60000, 10))

whitespace_test_matrix = whitespace_test_matrix.astype('float32')
whitespace_train_matrix = whitespace_train_matrix.astype('float32')

In [67]:
from keras import models
from keras import layers

# input1 = layers.Input(shape=(28 * 28,))
# x1 = layers.Dense(512, activation='relu')(input1)
# input2 = layers.Input(shape=(10,))
# x2 = layers.Dense(512, activation='relu')(input2)
# equivalent to added = keras.layers.add([x1, x2])
# added = layers.Add()([x1, x2])

network = models.Sequential()

network.add(layers.Dense(512, activation='relu', input_shape=(28 * 28,)))
network.add(layers.Dense(512, activation='relu', input_shape=(10,)))
#network.add(added)
network.add(layers.Dense(10, activation='softmax'))
network.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_19 (Dense)             (None, 512)               401920    
_________________________________________________________________
dense_20 (Dense)             (None, 512)               262656    
_________________________________________________________________
dense_21 (Dense)             (None, 10)                5130      
Total params: 669,706
Trainable params: 669,706
Non-trainable params: 0
_________________________________________________________________


In [81]:
import keras
from keras.layers import Input, Embedding, LSTM, Dense
from keras.models import Model

main_input = Input(shape=(28*28,), dtype='float32', name='main_input')
x = Embedding(output_dim=512, input_dim=60000, input_length=784)(main_input)
lstm_out = LSTM(128)(x)

auxiliary_output = Dense(1, activation='softmax', name='aux_output')(lstm_out)

auxiliary_input = Input(shape=(10,), name='aux_input')
x = keras.layers.concatenate([lstm_out, auxiliary_input])

x = Dense(128, activation='relu')(x)

main_output = Dense(1, activation='softmax', name='main_output')(x)

model = Model(inputs=[main_input, auxiliary_input], outputs=[main_output, auxiliary_output])

#model.compile(optimizer='rmsprop',
#                loss='categorical_crossentropy',
#                metrics=['accuracy'])

#model.fit([train_labels, whitespace_train_matrix], [train_labels, train_labels],
#          epochs=10, batch_size=128)

model.compile(optimizer='rmsprop',
              loss={'main_output': 'binary_crossentropy', 'aux_output': 'binary_crossentropy'},
              loss_weights={'main_output': 1., 'aux_output': 0.2})

# And trained it via:
model.fit({'main_input': train_labels, 'aux_input': whitespace_train_matrix},
          {'main_output': train_labels, 'aux_output': train_labels},
          epochs=50, batch_size=128)

ValueError: ignored

In [0]:
network.compile(optimizer='rmsprop',
                loss='categorical_crossentropy',
                metrics=['accuracy'])

In [71]:
epochs = 10
history = network.fit([train_images,
                      whitespace_train_matrix],
                      train_labels, 
                      epochs=epochs, 
                      batch_size=128, 
                      validation_data=(test_images, test_labels))

ValueError: ignored

In [8]:
epochs = 10
history = network.fit(train_images, 
                      train_labels, 
                      epochs=epochs, 
                      batch_size=128, 
                      validation_data=(test_images, test_labels))

Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 6s 99us/step - loss: 0.0075 - acc: 0.9980 - val_loss: 0.0838 - val_acc: 0.9789
Epoch 2/10
60000/60000 [==============================] - 6s 98us/step - loss: 0.0059 - acc: 0.9984 - val_loss: 0.0803 - val_acc: 0.9810
Epoch 3/10
60000/60000 [==============================] - 6s 98us/step - loss: 0.0044 - acc: 0.9990 - val_loss: 0.0894 - val_acc: 0.9807
Epoch 4/10
60000/60000 [==============================] - 6s 99us/step - loss: 0.0039 - acc: 0.9991 - val_loss: 0.0786 - val_acc: 0.9824
Epoch 5/10
60000/60000 [==============================] - 6s 98us/step - loss: 0.0034 - acc: 0.9990 - val_loss: 0.0848 - val_acc: 0.9807
Epoch 6/10
60000/60000 [==============================] - 6s 99us/step - loss: 0.0024 - acc: 0.9994 - val_loss: 0.0831 - val_acc: 0.9835
Epoch 7/10
60000/60000 [==============================] - 6s 96us/step - loss: 0.0020 - acc: 0.9995 - val_loss: 0.0828 - val_acc

In [11]:
history_dict = history.history
test_acc_values = history_dict['val_acc']
#prints out the last test value after the 10 testing epochs
print(test_acc_values[epochs - 1])

0.9825
